# Laboratorio 3: Problema de tiempo mínimo y lineal cuadrático

**Nombres:** Sebastián Acuña U. y Patricio Yañez A. <br>
**Fecha:** 4 de octubre de 2024 <br>
**Profesor:** Héctor Ramírez C. <br>
**Auxiliar:** Diego Olguín W. <br>
**Ayudantes:** Carlos Antil C. y Luis Fuentes C. <br>
**Curso:** [MA4703-1] Control Óptimo: Teoría y Laboratorio

In [1]:
# Librerías a utilizar

#using Pkg
#Pkg.add("DifferentialEquations")
#Pkg.add("JuMP")
#Pkg.add("NonlinearSolve")
#Pkg.add("OptimalControl")
#Pkg.add("LaTeXStrings")
#Pkg.add("NLPModelsIpopt")
#Pkg.add("ControlSystems")
#Pkg.add("Interpolations")

using LinearAlgebra
using Plots
using LaTeXStrings
using JuMP
using Ipopt
using NLPModelsIpopt
using DifferentialEquations
using NonlinearSolve
using OptimalControl
using ControlSystems
using Interpolations

## Parte A: Control de un carro-cohete en tiempo mínimo y método de resolución directo

### Ejercicio 1

Tenemos la ecuación de la dinámica

$$
\ddot{x} = -\alpha x + u
$$

con condiciones de borde $\mathbf{x_0} = (x_0, v_0) = (0,0)$ y $\mathbf{x_{t_f}} = (x_{t_f}, v_{t_f}) = (x_f, 0)$. Nos interesa resolver el problema 

$$
\min_{u \in \mathcal{A}}{t_f}; \ \ x(t_f;u,\mathbf{x_0}) = x_f; \ \ v(t_f;u,\mathbf{x_0}) = 0
$$

donde $|u(t)| \leq 1, \; \forall t$. 

Definiendo $v = \dot{x}$ se genera un sistema de ecuaciones cuya forma matricial es

$$
\dot{\begin{pmatrix}
x(t) \\
v(t)
\end{pmatrix}}
=
\begin{pmatrix}
0 & 1 \\
-\alpha & 0
\end{pmatrix}
\begin{pmatrix}
x(t) \\
v(t)
\end{pmatrix}
+
\begin{pmatrix}
0 \\
1
\end{pmatrix}
u(t)
$$

Si discretizamos la dinámica particionando el intervalo $[0,t_f]$ en $N$ puntos $0 = t_1 < t_2 < \cdots < t_{N} = t_f$, donde $t_i = (i-1) \dfrac{t_f}{N-1}$ obtenemos el siguiente sistema 

\begin{align*}
\dfrac{x_{i+1}-x_i}{h} &= v_i \\
\dfrac{v_{i+1}-v_i}{h} &= -\alpha x_i + u_i
\end{align*}

donde $x_i = x(t_i), v_i = v(t_i), u_i = u(t_i)$ y $h = \frac{t_f}{N-1}$. Esto en particular implica que 

$$
\begin{pmatrix}
x_{i+1} \\
v_{i+1}
\end{pmatrix} = \begin{pmatrix}
x_i \\
v_i
\end{pmatrix} + h \begin{pmatrix}
v_i \\
-\alpha x_i
\end{pmatrix} + h \begin{pmatrix}
0 \\
u_i
\end{pmatrix}
$$

Con ello, el problema de optimización consiste en minimizar $t_f$ bajo las restricciones provistas por la dinámica discretizada, el valor de $h$ como *step function* y las condiciones de borde mencionadas al inicio.

### Ejercicio 2

In [ ]:
# Definición de la discretización de Euler y resolución utilizando JuMP

function euler_discretization(N, α, x_f)
    model = JuMP.Model(Ipopt.Optimizer)
    
    @variable(model, 0 <= t_f)
    @variable(model, -1 <= u[1:N-1] <= 1)
    @variable(model, x[1:N])
    @variable(model, v[1:N])
    
    h = t_f / (N - 1)
    
    @constraint(model, x[1] == 0)
    @constraint(model, v[1] == 0)
    @constraint(model, x[N] == x_f)
    @constraint(model, v[N] == 0)
    
    for i in 1:N-1
        @constraint(model, x[i+1] == x[i] + h * v[i])
        @constraint(model, v[i+1] == v[i] + h * (-α * x[i] + u[i]))
    end
    
    @objective(model, Min, t_f)
    
    optimize!(model)
    
    return value.(x), value.(v), value.(u), value(t_f)
end

# Parámetros de juguete
N = 100
α = 1.0
x_f = 100.0

# Resolución
x, v, u, t_f = euler_discretization(N, α, x_f)

In [ ]:
# Resultados
println("Tiempo final óptimo: ", t_f)
println("Posiciones: ", x)
println("Velocidades: ", v)
println("Control: ", u)

A continuación se prueba el método discreto para distintos valores de $N$, $\alpha$ y la posición final $x_f$.

In [ ]:
# N = 10
N = 10
α_values = [0, 1, 5]
x_f_values = [-20, 10, 40]

# Resolver el problema de tiempo mínimo discretizado probando las combinaciones de N, α, x_f
results_1 = []
for a in α_values
    for xf in x_f_values
        try
            x, v, u, t_f = euler_discretization(N, a, xf)
            push!(results_1, (N=N, α=a, x_f=xf, t_f=t_f, x=x, v=v, u=u))
            println("N: $N, α: $a, x_f: $xf, t_f: $t_f")
        catch e
            println("Error con N: $N, α: $a, x_f: $xf - ", e)
        end
    end
end

In [ ]:
results_1

In [ ]:
# N = 100
N = 100
α_values = [0, 1, 5]
x_f_values = [-20, 10, 40]

# Resolver el problema de tiempo mínimo discretizado probando las combinaciones de N, α, x_f
results_2 = []
for a in α_values
    for xf in x_f_values
        try
            x, v, u, t_f = euler_discretization(N, a, xf)
            push!(results_2, (N=N, α=a, x_f=xf, t_f=t_f, x=x, v=v, u=u))
            println("N: $N, α: $a, x_f: $xf, t_f: $t_f")
        catch e
            println("Error con N: $N, α: $a, x_f: $xf - ", e)
        end
    end
end

In [ ]:
results_2

In [ ]:
# N = 250
N = 250
α_values = [0, 1, 5]
x_f_values = [-20, 10, 40]

# Resolver el problema de tiempo mínimo discretizado probando las combinaciones de N, α, x_f
results_3 = []
for a in α_values
    for xf in x_f_values
        try
            x, v, u, t_f = euler_discretization(N, a, xf)
            push!(results_3, (N=N, α=a, x_f=xf, t_f=t_f, x=x, v=v, u=u))
            println("N: $N, α: $a, x_f: $xf, t_f: $t_f")
        catch e
            println("Error con N: $N, α: $a, x_f: $xf - ", e)
        end
    end
end

In [ ]:
results_3

#### Selección de gráficos de posición y velocidad obtenidos

Gráfico para $N = 10$, $\alpha = 0$, $x_f = -20$

In [ ]:
result1 = results_1[1]

# Extraer los valores de x, v, u y t_f
x = result1.x
v = result1.v
u = result1.u
t_f = result1.t_f
N = result1.N

# Crear el rango de tiempo
t = range(0, stop=t_f, length=N)

In [ ]:
# Graficar la trayectoria x(t)
plot(t, x, label="Posición x(t)", xlabel="Tiempo t", ylabel="Posición x", title="Trayectoria óptima para N=$N, α=$(result1.α), x_f=$(result1.x_f)")

In [ ]:
# Graficar la velocidad v(t)
plot(t, v, label="Velocidad v(t)", xlabel="Tiempo t", ylabel="Velocidad v", title="Velocidad óptima para N=$N, α=$(result1.α), x_f=$(result1.x_f)")

In [ ]:
# Graficar el control u(t)
plot(t[1:N-1], u, label="Control u(t)", xlabel="Tiempo t", ylabel="Control u", title="Control óptimo para N=$N, α=$(result1.α), x_f=$(result1.x_f)", legend=:bottomright)

### Ejercicio 3

In [ ]:
# Definición y resolución del problema de tiempo mínimo utilizando OptimalControl

function solve_min_time_continuous(α, x_f)
    model = OptimalControl.Model(Ipopt.Optimizer, 2, 1)

    @def begin
        t_f ∈ R, variable
        t ∈ [0, t_f], time
        x ∈ R², state
        u ∈ [-1, 1], control
        x' = [x[2], -α * x[1] + u]
        x(0) = [0, 0]
        x(t_f) = [x_f, 0]
        t_f >= 0
    end

    @objective(model, Min, tf)

    optimize!(model)

    return solution(model)
end

# Parámetros de juguete
α = 1.0
x_f = 100.0

# Resolución
sol = solve_min_time_continuous(α, x_f)

println("Tiempo final óptimo: ", sol.tf)
println("Posiciones: ", sol.x[1,:])
println("Velocidades: ", sol.x[2,:])
println("Control: ", sol.u[1,:])

### Ejercicio 4

## Parte B: Problema lineal cuadrático. Carro cohete en 2-D

### Ejercicio 5

### Ejercicio 6

### Ejercicio 7

### Ejercicio 8